# Analyse des temps de paroles des élections europeenes

Preparations des fichiers pour affichage.<br>
<br>
Le site de l'arcom nous fournie plusieurs fichiers à l'adresse suivante : <br>
<a href = "https://www.arcom.fr/actualites/elections-europeennes-2024-publication-des-releves-des-temps-de-parole"> page arcom </a><br>


In [1]:
# import des bibliotheques
import pandas as pd
import os


In [2]:
#lister les nom des fichiers dans les données brutes
liste_fichier =os.listdir('donnee_brut')


In [4]:
# faire les trucs pour tout les fichiers :
temps_parole_EU2024 = pd.DataFrame()
for element in liste_fichier:
    # lire le fichier 
    tempo_df = pd.read_csv('donnee_brut/'+element,encoding='latin',sep=';',header=3)
    # garder les 2 derniere ligne du temps presidentiel
    df_president = tempo_df.tail(2)
    # supprimer les ligne du temps presidentiel
    tempo_df = tempo_df[:-2]
    # rajouter une ligne pour le temps presidentiel 
    tempo_df.loc[len(tempo_df)]=['temps_president',df_president.iat[1,1]]
    # rajouter le nom de la chaine
    tempo_df['Nom du media'] = (element[30:-4])
    # mettre le temps total du media en colonne
    #total = tempo_df.loc[tempo_df['Listes candidates']=='Total des temps de parole','Total période (durée)'].to_string()
    #tempo_df['temps total'] = total[-8:]
    tempo_df = tempo_df[tempo_df['Listes candidates']!='Total des temps de parole']
    # concat les dataframe entre eux
    temps_parole_EU2024 = pd.concat([temps_parole_EU2024,tempo_df])
    

Europeennes-2024_Temps-parole_BFMBusiness.csv
Europeennes-2024_Temps-parole_BFMTV.csv
Europeennes-2024_Temps-parole_C8.csv
Europeennes-2024_Temps-parole_Cnews.csv
Europeennes-2024_Temps-parole_Euronews.csv
Europeennes-2024_Temps-parole_Europe1.csv
Europeennes-2024_Temps-parole_France2.csv
Europeennes-2024_Temps-parole_France24.csv
Europeennes-2024_Temps-parole_France3.csv
Europeennes-2024_Temps-parole_France5.csv
Europeennes-2024_Temps-parole_FranceCulture.csv
Europeennes-2024_Temps-parole_FranceInfo.csv
Europeennes-2024_Temps-parole_FranceInter.csv
Europeennes-2024_Temps-parole_LCI.csv
Europeennes-2024_Temps-parole_M6.csv
Europeennes-2024_Temps-parole_RFI.csv
Europeennes-2024_Temps-parole_RMC-Story.csv
Europeennes-2024_Temps-parole_RMC.csv
Europeennes-2024_Temps-parole_RMCDecouverte.csv
Europeennes-2024_Temps-parole_RTL.csv
Europeennes-2024_Temps-parole_RadioClassique.csv
Europeennes-2024_Temps-parole_SudRadio.csv
Europeennes-2024_Temps-parole_TF1.csv
Europeennes-2024_Temps-parole_TMC

ParserError: Passed header=3 but only 1 lines in file

In [ ]:
couleur_pol = pd.read_csv('donnee_brut/couleurs.csv',encoding='latin')

Liste des medias contenante les fichiers :<br>
Canal plus n'a pas fourni de temps de paroles<br>

In [ ]:
print(temps_parole_EU2024['Nom du media'].unique())

In [ ]:
# listez les media en fonction de leur categories
liste_tv_gen = ['TMC', 'France2', 'France3','M6', 'RMC-Story',
       'TF1', 'France5', 'C8', 'RMCDecouverte', 'TV5Monde',
       ]

liste_tv_info = ['LCI', 'franceinfo_',
       'BFMTV', 'France24', 'Euronews', 'Cnews']

liste_radio = ['RTL', 'RFI', 'FranceInfo', 'FranceCulture',
       'SudRadio', 'RMC', 'Europe1', 'BFMBusiness',
       'RadioClassique', 'FranceInter']

# ajouter les categories dans le dataframe :
temps_parole_EU2024['Type de media'] = 'inconnu'

for media in liste_tv_gen: 
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'TV généraliste'
for media in liste_tv_info:
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'TV info continue' 
for media in liste_radio:
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'radio' 

Catégorisation des médias : <br>

In [ ]:
temps_parole_EU2024['Type de media'].unique()

In [ ]:
temps_parole_EU2024.info()

In [ ]:
# voir pour changer le type de données de la periode tout convertir en seconde puis en heure
temps_parole_EU2024['heure'] = temps_parole_EU2024['Total période (durée)'].str[:2]
temps_parole_EU2024['minute'] = temps_parole_EU2024['Total période (durée)'].str[3:5]
temps_parole_EU2024['seconde'] = temps_parole_EU2024['Total période (durée)'].str[-2:]

temps_parole_EU2024['heure'] = temps_parole_EU2024['heure'].astype(int)
temps_parole_EU2024['minute'] = temps_parole_EU2024['minute'].astype(int)
temps_parole_EU2024['seconde'] = temps_parole_EU2024['seconde'].astype(int)

temps_parole_EU2024['Durée en seconde'] = temps_parole_EU2024['heure']*3600 + temps_parole_EU2024['minute']*60 +temps_parole_EU2024['seconde']
temps_parole_EU2024['Durée en minute'] = temps_parole_EU2024['Durée en seconde']/60 

In [ ]:
temps_parole_EU2024.drop(columns=['heure','minute','seconde'],inplace=True)

# Temps de parole en secondes des listes :<br>

In [ ]:
temps_parole_EU2024.groupby(by='Listes candidates').agg({'Durée en seconde':'sum'}).reset_index()

In [ ]:
#calcul du temps total :

df_temps_total = temps_parole_EU2024.groupby(by='Nom du media').agg({'Durée en seconde':'sum'}).reset_index().rename(columns={'Durée en seconde':'Temps total'})



In [ ]:
temps_parole_EU2024 = pd.merge(temps_parole_EU2024,df_temps_total,on='Nom du media')
temps_parole_EU2024 = pd.merge(temps_parole_EU2024,couleur_pol,on='Nom du media')

In [ ]:
couleur_pol

In [ ]:
temps_parole_EU2024.to_csv('donnee_contruite/temps_paroles_EU2024.csv',index=False)